In [27]:
import pandas as pd
from langdetect import detect
from SPARQLWrapper import SPARQLWrapper, JSON
import re
import numpy as np
import sys
import os
import time
from nltk.tag import pos_tag
import urllib.parse

In [28]:
target = pd.read_csv("D:\Dokumenty\Zastosowanie Technologii Informatycznych\data\CEA_Round1_Targets.csv",
                    sep = ',',
                     skiprows=range(112,113), # 112 wadliwy wiersz
                    names=['TableID', 'ColumnID', 'RowID'])
target[:3]

,TableID,ColumnID,RowID
0,50245608_0_871275842592178099,0,154
1,39107734_2_2329160387535788734,1,32
2,22864497_0_8632623712684511496,0,227


In [29]:
def replace_characters(chars_to_replace_list_of_dicts, cell_contents_list, cell_content):
    # zamienia wszystkie znaki odpowiadające kluczowi np. '[?]' na to co w wartości ''
    # każdy znak zamienia osobno. Nie wyszukuje stringa '[?]', tylko każdy znak po kolei szukam i zamieniam
    for dicts in chars_to_replace_list_of_dicts:
        for old_chars, new_char in dicts.items():
            for old_char in old_chars:
                cell_content = cell_content.replace(old_char, new_char)
            if cell_content not in cell_contents_list:
                cell_contents_list.append(cell_content)
    return cell_contents_list

def create_queries(cell_content):
    cell_content_copy = cell_content
    cell_contents_list = []
    cell_contents_list.append(cell_content) # oryginalne
    cell_contents_list.append(str('The ' + cell_content)) # opcja z The na początku
    cell_contents_list.append(cell_content.capitalize())
    cell_contents_list.append(cell_content.title())
    
    # pojawiały się wartości typy "Guardian The", a powinno być "The Gurdian", 
    # albo "Thunderbird, Lake", a powinno być "Lake Thunderbird"
    if re.search(", [a-zA-Z]*$", cell_content_copy):
        found_string = re.search(", [a-zA-Z]*$", cell_content_copy).group(0)
        new_cell_content = cell_content_copy.replace(found_string, '')
        found_string = found_string.replace(', ','')
        new_cell_content = found_string.capitalize() + ' ' + new_cell_content
        
        cell_contents_list.append(new_cell_content)
    
    # pojawiały się wartości typy "Guardian The", a powinno być "The Gurdian" to samo z A i An
    if re.search(" The$", cell_content):
        index = cell_content.find(" The")
        cell_contents_list.append("The " + cell_content[:index].replace(',','').strip())
        cell_contents_list.append(cell_content[:index].strip())
    elif re.search(" A$", cell_content):
        index = cell_content.find(" A")
        cell_contents_list.append("A " + cell_content[:index].replace(',','').strip())
        cell_contents_list.append(cell_content[:index].strip())
    elif re.search(" An$", cell_content):
        index = cell_content.find(" An")
        cell_contents_list.append("An " + cell_content[:index].replace(',','').strip())
        cell_contents_list.append(cell_content[:index].strip())

    # usunięcię The z początku
    if re.search("^The ", cell_content):
        cell_contents_list.append(re.sub('^The ', '', cell_content))


    strings_to_replace_dict = [{'[?]':''}, {',':''}, {'-':' '}, {'?':''}] # dict = {chars_to_replace:new_char}
    cell_contents_list = replace_characters(strings_to_replace_dict, cell_contents_list, cell_content)
    
    list_of_queries = []
    # pierwsze zapytanie - bierze zawartość sformatowanej komórki i wyszukuje linku,
    query = 'SELECT ?resource WHERE { ?resource rdfs:label ' + '"' + str(cell_content) + '"' + "@en" + '}'
    list_of_queries.append(query)

    for cell_content in cell_contents_list:
        query = 'SELECT ?resource WHERE { ?resource rdfs:label ' + '"' + str(cell_content) + '"' + "@en" + '} '
        list_of_queries.append(query)


    lang = 'en'
    try:
        lang = detect(cell_content)
    except:
        pass

    if lang != 'en':
        for cell_content  in cell_contents_list:
            query = 'SELECT ?resource WHERE { ?resource rdfs:label ' + '"' + str(cell_content) + '"' + '@' + str(lang) + '} '
            list_of_queries.append(query)
    
    return list_of_queries

def create_queries_for_films(cell_content):
    if len({'Title', 'Year'}.intersection(set(df.columns))) == 2:
        row = df[row_id-1:row_id]
        film_year = row['Year'].values[0]

        cell_content = cell_content.replace(',','')
        cell_content = cell_content.replace('The','')
        cell_content = cell_content.replace(',','')
        cell_content = cell_content.strip()


        query1 = 'SELECT ?resource WHERE {' + \
                    '?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"^' + str(cell_content) + '$", "i") . ' + \
                    '?resource rdf:type dbo:Film . }'

        query2 = 'SELECT ?resource WHERE {' + \
                    '?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"^The ' + str(cell_content) + '$", "i") . ' + \
                    '?resource rdf:type dbo:Film . }'

        query3 = 'SELECT ?resource WHERE {' + \
                    '?resource rdfs:label '  + '"' + str(cell_content) + '" . ' + \
                    '?resource rdf:type dbo:Film . ' + \
                    '?redirect dbo:wikiPageRedirects ?resource FILTER regex(?redirect, '  + '"' + str(film_year) + '") . } '

        query4 = 'SELECT ?resource WHERE {' + \
                    '?resource rdfs:label '  + '"' + str(cell_content) + '" . ' + \
                    '?resource rdf:type dbo:Film . ' + \
                    '?resource rdfs:label ?name2 FILTER regex(?name2, '  + '"' + str(film_year) + '") . }' 

        query5 = 'SELECT ?resource WHERE {' + \
                    '?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"' + str(cell_content) + '", "i") . ' + \
                    '?resource rdf:type dbo:Film . ' + \
                    '?redirect dbo:wikiPageRedirects ?resource FILTER regex(?redirect, '  + '"' + str(film_year) + '") . } '

        query6 = 'SELECT ?resource WHERE {' + \
                    '?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"' + str(cell_content) + '", "i") . ' + \
                    '?resource rdf:type dbo:Film . ' + \
                    '?resource rdfs:label ?name2 FILTER regex(?name2, '  + '"' + str(film_year) + '") . }'

        query7 = 'SELECT ?resource WHERE {' + \
                    '{?resource rdfs:label ?name1 FILTER regex(?name1,'  + '"' + str(cell_content) + '", "i") . ' + \
                    '?resource rdf:type dbo:Film . }}'

        list_of_queries = [query1, query2, query3, query4, query5, query6, query7]
        
        return list_of_queries
    else:
        return []

def send_query(query):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # jeśli wystąpi błąd z połączeniem się z serwerem to próbujemy aż do skutku
    results = None
    while results is None:
        try:
            results = sparql.query().convert()
        except Exception as e:
#                 print("EXCEPTION:" + str(e))
#                 print("TAKI: " + str(cell_content))
            continue
    return results

def get_URI_from_results(results, query, verbose=0):
    # jeśli nic nie znaleziono no to trudno, nie mamy linku, wstawiamy NOT FOUND URI
#     if cell_content_copy == 'Ruppell\'s griffon vulture':
#         print(query)
    
    if str(results['results']['bindings']).find("http://dbpedia.org/resource/") == -1:
        # printy przydatne do sprawdzania jakich zawartości nam nie wyszukał,
        # trzeba analizowac i dodawać kolejne warunki, zmieniać cell_content na różne sposoby
        if verbose:
            print(table_id, column_id, row_id)
            print(cell_content_copy)
            print(query)    
        return 'NOT FOUND URI'


    # jeśli jednak mamy jakies linki to wyciągamy go z jsona,
    # linków może być więcej niż 1, aktualnie bierzemy pierwszy lepszy
    results_list = results['results']['bindings']
    for result_dict in results_list:
        uri = result_dict['resource']['value']
        match = re.match("http://dbpedia.org/resource/*", uri)

        # czasami pojawiają dziwne linki np. http://dbpedia.org/resource/Category:Max_Weber
        # my takich nie chcemy i musimy ich unikać
        # interesują nas tylko np.  http://dbpedia.org/resource/Max_Weber
        # więc to jest warunek pozbywajacy się wszystkich linków z Category:
        if match and uri[5:].find("Category:") == -1:
#                 answers_dict['URI'].append(str(uri)
#                 formatted_uri = 'http://dbpedia.org/resource/' + str(urllib.parse.quote(uri[28:]))
#                 formatted_uri = formatted_uri.replace('-','%E2%80%93')
            return uri
    
    return 'NOT FOUND URI'

In [30]:
start = time.time()

# słownik, z którego będzie tworzony wynikowy df
answers_dict = {"TableID":[],
               "ColumnID":[],
               "RowID":[],
               "URI":[]}

# zmienna, aby ograniczyć liczbę przetwarzanych wierszy, niżej masz 3 linijki skomentowane,
# odkomentowujesz je i ustawiasz sobię liczbę wierszy w tym warunku, aktualnie 1000
counter = 0
# zlicza ile linków nie zostało znalezionych w ogóle,
# tzn. po wysłaniu zapytania nie dostaliśmy żądnej odpwiedzi od sparql (wiersze oznaczone przez NOT FOUND URI)
not_found_uris = 0

for index, row in target.iterrows():
    try:
        counter = counter + 1
#         if counter == 100:
#             break


        table_id = row['TableID']
        column_id = row['ColumnID']
        row_id = row['RowID']
        
        # wczytanie odpowiedniej tabeli
        df = pd.read_csv(f"D:\Dokumenty\Zastosowanie Technologii Informatycznych\data\CEA_Round1\{table_id}.csv",
                    sep=',')
        
            
        # wyciągniecie wartości komórki, która nas interesuje
        cell_content = df.iloc[row_id-1:row_id, column_id:(column_id+1)].values[0][0]
        cell_content_copy = cell_content
        
        print(str(counter) + ' ' + cell_content)
        
        # usunięcie białych znaków z początku i końca stringa
        cell_content = cell_content.strip()
        
        list_of_queries = []
        list_of_queries += create_queries_for_films(cell_content)
        list_of_queries = list_of_queries + create_queries(cell_content)
        
        for id, query in enumerate(list_of_queries):
            results = send_query(query)                              
            uri = get_URI_from_results(results, query, verbose=0)

            if uri != 'NOT FOUND URI':             
                answers_dict['TableID'].append(table_id)
                answers_dict['ColumnID'].append(column_id)
                answers_dict['RowID'].append(row_id)
                answers_dict['URI'].append(uri)
                break
            elif id+1 == len(list_of_queries) and uri == 'NOT FOUND URI':
                answers_dict['TableID'].append(table_id)
                answers_dict['ColumnID'].append(column_id)
                answers_dict['RowID'].append(row_id)
                answers_dict['URI'].append("NOT FOUND URI")
                not_found_uris += 1
                
                print(table_id, column_id, row_id)
                print(cell_content_copy)
                
                    

        # aktualnie nie używane, ale pewnie będzie
        # pos_tag() - kategoryzuje słowa, np. znajduje czasowniki, rzeczowniki, nazwy własne (jak imiona i nazwiska)
        # chodzi o to, że czasami odpowiedzi są w angielskiej dbpedii,
        # np. Alex Rodriguez wykrywa jako język ca - pewnie kataloński, 
        # ale nie ma jego odpowiednika w katalońskiej dbpedii, a w angielskiej jest
        # dlatego chcę wykrywać nazwy wkłasne jak imiona i nazwiska i wtedy zawsze szukać w angielksiej dbpedii,
        # ale z tym też był problem czasami i na razie nie myślałem nad tym dalej
        # NNP - to kategoria nazwy własnej, np. imię
#         tags = pos_tag(cell_content.split())
#         is_one_NNP = False
#         for tag in tags:
#             if tag[1] == "NNP":
#                 is_one_NNP = True
#                 break
#         try:
#             if is_one_NNP:
#                 lang = '@en'
#             else:
#                 lang = str('@') + str(detect(cell_content))
#         except Exception:
#             lang = ''

#         list_of_queries = create_queries(cell_content)
#         for query in list_of_queries:
#             results = send_query(query)                              
#             uri = get_URI_from_results(results, query, verbose=1) 
                         
#             answers_dict['TableID'].append(table_id)
#             answers_dict['ColumnID'].append(column_id)
#             answers_dict['RowID'].append(row_id)
#             answers_dict['URI'].append(uri)
#             if uri == 'NOT FOUND URI':
#                 not_found_uris += 1
#                 continue

    except Exception as e:
        # na początku były problemy z danymi, naprawiłem
        # potem zaczęły pojawiać sę problemy, żę serwer czasami nie odpowiadał,
        # więc jak jest error to olewamy i przechodzimy do następnego wiersza
        # będzie trzeba zrobić, że jak serwer nie odpowie z jakiegoś powodu to trzeba ponowić to samo zapytanie
        # zamiast pomijać, bo tracimy wiersze
        exc_type, exc_obj, exc_tb = sys.exc_info()
        print(table_id, column_id, row_id) # jaki plik, kolumna i wiersz
        print(cell_content_copy) # zawartość komórki
        print(e) # jaki błąd, teraz chyba tylko z brakiem połączenia z serwerem
        print(exc_tb.tb_lineno) # w jakiej liniii błąd
        continue



# print(answers_dict)
result_df = pd.DataFrame(answers_dict)
end = time.time()
print(end-start)
print(not_found_uris)

1 Pearl Harbor
2 Minneapolis-Saint Paul International Airport
3 Need for Speed: Carbon
4 Alex Rodriguez
5 King Kong
6 School of Rock , The
7 Lex Luger
8 Crash Bash
9 The Hurt Locker
10 Lion
11 Lac-Seul
12 Fear and Loathing in Las Vegas
13 Max Weber
14 Kidsty Pike
15 Randy Johnson
16 TimeShift
17 Batman: Arkham City
18 Dazzy Vance?
19 A Clockwork Orange
20 Bill Collins
21 Greenland
22 Thunbergia grandiflora
23 Peter of Sebaste[?]
24 Jimmy Ryan
25 State Hermitage Museum
26 Mario Party 8
27 Lygodium palmatum
28 New Taiwan Dollar
29 Mary Poppins
30 The Curse of Monkey Island
31 Super Mario All-Stars + Super Mario World
32 Ruppell's griffon vulture
13719111_1_5719401842463579519 0 48
Ruppell's griffon vulture
33 Contender
34 Common fiscal
35 Heat
36 Pope Linus
37 Broken Bow Lake
38 TREE SWALLOW
39 The Wild Bunch
40 FIFA 08
41 The Legend of Zelda: Ocarina of Time
42 Fellbarrow
43 Sim City
44 Asparagus densiflorus
45 To Be or Not to Be
46 John of the Cross
47 Mark Langston
48 Soldier of Fortu

326 Eppa Rixey?
327 Senna obtusifolia
328 Royal Academy of Arts
329 Solanum capsicoides
330 Sporobolus compositus
331 BioShock
332 FC Instincts Predator
22864497_0_8632623712684511496 0 139
FC Instincts Predator
333 Super R-Type
334 Pennisetum ciliare
335 Animal Crossing: Wild World
336 11:14
337 Joschka Fischer
338 L.A. Confidential
339 Pope Anacletus
340 Over the Hedge
341 In Living Color
342 Pope Nicholas I
343 Boogie Nights
344 Casino Royale
345 Snatch.
346 Pope Sergius I
347 Dillenia suffruticosa
348 Star Wars: Episode II - Attack Of The Clones
54719588_0_8417197176086756912 3 130
Star Wars: Episode II - Attack Of The Clones
349 Star Wars
350 Joy Ride
351 Mordecai Brown?
352 Tullberg's woodpecker
353 LEGO Star Wars II: The Original Trilogy
354 Egypt
355 Assassin's Creed: Brotherhood
356 Interview
357 Deja Vu
358 Logan International Airport
359 Biochemical Journal
360 Christy Mathewson?
361 Spider-Man
362 Deliverance
363 Eragrostis curvula
364 Basil the Great
365 Out Cold
366 The B

14067031_0_559833072073397908 0 53
Zambabwe
665 Gregory the Illuminator
666 Producers, The
667 Gro?er-Sklavensee
8468806_0_4382447409703007384 1 10
Gro?er-Sklavensee
668 Jackass
669 Icarus, International Journal of Solar System Studies
670 O' the Pines, Lake (updated on 06/07/2004)
21245481_0_8730460088443117515 1 183
O' the Pines, Lake (updated on 06/07/2004)
671 Stephen
672 White-rumped seedeater
673 Waking The Dead
674 Madden NFL 2006
675 PURPLE FINCH
676 Gem?ldegalerie, Staatliche Museen zu Berlin
9834884_0_3871985887467090123 1 64
Gem?ldegalerie, Staatliche Museen zu Berlin
677 The Legend of Zelda: The Minish Cap
678 Statens Museum for Kunst
679 Dark chanting-goshawk
680 CSI: Crime Scene Investigation
681 WHITE-FOOTED MOUSE
682 Bionic Commando
683 Lord of War
684 TUFTED TITMOUSE
685 Donkey Kong Country
686 Carrefour
687 The Legend of Zelda: Phantom Hourglass
688 Australian Dollar
689 QUEEN SNAKE
690 Terminator 2: Judgment Day
691 Carex geyeri
692 Chhatrapati Shivaji Maharaj Vastu 

953 Sesbania grandiflora
954 BRIGID of Ireland
955 Dr. Strangelove
956 Cy Williams
957 Fight Night 2004
958 Allard Pierson Museum
959 Sergeant?s Crag
99070098_0_2074872741302696997 1 140
Sergeant?s Crag
960 Volcano
961 Some Like It Hot
962 YELLOW-RUMPED WARBLER
963 WHITE-BREASTED NUTHATCH
964 Raising Arizona
965 Frank Robinson
966 Nicholas of Myra
967 Resident Evil 4
968 Carex saxatilis
969 Metal Gear Solid
970 HP
971 Fort Loudoun Lake (updated 06/07/2004)
21245481_0_8730460088443117515 1 140
Fort Loudoun Lake (updated 06/07/2004)
972 Sunrise
973 Mus?e des Beaux-Arts de Rennes
9834884_0_3871985887467090123 1 214
Mus?e des Beaux-Arts de Rennes
974 NBA '09: The Inside
975 More Media >>
80588006_0_6965325215443683359 1 21
More Media >>
976 God of War
977 Kuwait
978 The Lord of the Rings: The Two Towers
979 WWE All Stars
980 Namco Museum 64
981 Sonic Adventure 2 Battle
982 Calamagrostis canadensis
983 Scoparia dulcis
984 Blue-headed coucal
985 Leucopoa kingii
986 Anagallis arvensis
987 The

22864497_0_8632623712684511496 0 102
Darkness, TheThe Darkness
1259 Holbrook, Lake * Also includes Lake Fork Reservoir, TX and Lake Quitman, TX
21245481_0_8730460088443117515 1 168
Holbrook, Lake * Also includes Lake Fork Reservoir, TX and Lake Quitman, TX
1260 Sergius
1261 The Rules of the Game
1262 F-Zero X
1263 Vertigo
1264 Lord of the Rings: The Two Towers
1265 George Burns
1266 Clusia rosea
1267 The Shining
1268 Averrhoa bilimbi
1269 Willy Wonka & the Chocolate Factory
1270 McQueeney, Lake *
21245481_0_8730460088443117515 1 176
McQueeney, Lake *
1271 Thank You for Smoking
1272 The Andy Griffith Show
1273 Eurasian marsh harrier
1274 Clyde, Lake
1275 Leave It to Beaver
1276 Martin of Tours
1277 Ghost Town
1278 Rocky
1279 ALLEGHENY WOODRAT
1280 Koeleria macrantha
1281 Holy See (Vatican City)
1282 The Princess Bride
1283 George Bush Intercontinental Airport
1284 Amadeus
1285 Ugetsu
1286 All in the Family
1287 CFP Franc
1288 Melinda and Melinda
1289 Mandrill
1290 Solomon Islands
1291 T

...

6918 Dazed and Confused
6919 Prey
6920 Tonga
6921 Black-casqued wattled hornbill
6922 Schindler's List
6923 Kleiner-Sklavensee
6924 Red-necked spurfowl
6925 Alban
6926 Cryptostegia grandiflora
6927 Summer Of Sam
6928 Pope John I
6929 Kwacha
6930 Mendota, Lake
6931 Airwolf
6932 Puerto Rico
6933 Stybarrow Dodd
6934 Hartsop Above How
6935 TIM
6936 John Climacus
6937 Wattled starling
6938 Madagascar: Escape 2 Africa
6939 Senecio madagascariensis
6940 Wipeout Fusion
6941 CFA Franc BCEAO
6942 Gone in Sixty Seconds
6943 Jordan
6944 Jordan
6945 Dark Knight, The
6946 SHORT-EARED OWL
6947 The Andy Griffith Show
6948 Tomb Raider: Underworld
6949 White-headed saw-wing
6950 Arkabutla Lake
6951 Big Lebowski, The
6952 Raise
6953 Twisted Metal
6954 Speckled pigeon
6955 Karaoke Revolution
6956 Gil Hodges
6957 Vermont
6958 Showgirls
6959 Bourne Ultimatum, The
6960 Benin
6961 Goldman Sachs
6962 Donkey Kong Country 3: Dixie Kong's Double Trouble!
6963 Children of Men
6964 New Zealand
6965 DOWNY WOODPECKER

7243 COTTON MOUSE
7244 Los Angeles County Museum of Art (LACMA)
9834884_0_3871985887467090123 1 8
Los Angeles County Museum of Art (LACMA)
7245 The Royal Tenenbaums
7246 Into the Wild
7247 Resistance: Fall of Man
7248 John Smoltz
7249 Deutsche Bank
7250 The Lady Eve
7251 Tim Wakefield (7)
50270082_0_444360818941411589 1 111
Tim Wakefield (7)
7252 Sam Mertes
7253 Tim Hudson (16)
50270082_0_444360818941411589 1 157
Tim Hudson (16)
7254 Gray Crag
7255 Tony Hawk Underground 2
7256 Fred Clarke
7257 Spaceballs
7258 Broad-billed roller
7259 WHIP-POOR-WILL
7260 Say It Isn't So
7261 Ansorge's greenbul
7262 Belgium
7263 SNOW GOOSE
7264 Spermacoce prostrata
7265 Harley-Davidson
7266 Grey crowned crane
7267 Green-backed woodpecker
7268 The Shining
7269 City Island
7270 Pope Eutychian
7271 Pope Felix II
7272 Is It Fall Yet?
7273 ING
7274 Dirty Harry
7275 Margarita
7276 Earth Defense Force 2017
7277 MotorStorm
7278 Costa Rica
7279 College Hoops 2K8
7280 Metal Gear Solid 3: Snake Eater
7281 Trackmani

7587 Thailand
7588 WALL STREET JOURNAL USA
9567241_0_5666388268510912770 1 11
WALL STREET JOURNAL USA
7589 Fight Night Round 4
7590 Tom Clancy's Rainbow Six: Vegas
7591 Alabama
7592 The Band's Visit
7593 Setaria palmifolia
7594 EASTERN PHOEBE
7595 Super Troopers
7596 Tour of Duty
7597 BLACK-NECKED STILT
7598 Pope Soter
7599 BACHMAN'S SPARROW
7600 Andujar Cedeno
7601 Tschanysee
8468806_0_4382447409703007384 1 66
Tschanysee
7602 West Bank
7603 Teylers Museum
7604 Ratchet & Clank 3
7605 Control
7606 Tenchu Z
7607 San Marino
7608 Libya
7609 Batman Returns
7610 Iron Man
7611 Danish Krone
7612 Men Of Honor
7613 The King of Queens
7614 NASCAR 08
7615 Ninian
7616 Qatar
7617 The Godfather: Part II
7618 Hyde Collection
7619 New Zealand Dollar
7620 Mus?e d'Art et d'Histoire
9834884_0_3871985887467090123 1 114
Mus?e d'Art et d'Histoire
7621 Scott Pilgrim vs. the World
7622 China
7623 Spyro The Dragon
7624 McPhee Reservoir
7625 Namibia
7626 Nosferatu
7627 M
7628 foxnews
7629 Dr. Horrible's Sing-Alo

7877 Pope Eleutherius
7878 Rescue Dawn
7879 Jaws
7880 Black Knight
7881 Tony Mullane
7882 Red Dead Redemption
7883 Physical Review Letters
7884 Finland
7885 Cyathea cooperi
7886 Terminator Salvation
7887 Tekken Tag Tournament
7888 Die Sims 3
91959037_0_7907661684242014480 1 398
Die Sims 3
7889 Dinamalar
7890 Barbara[?]
7891 Bahrain
7892 Startup.com
7893 Frasier
7894 Blade Runner
7895 Star Wars: The Force Unleashed II
7896 Gandhi
7897 Therese of Liseaux[?]
7898 Shutter Island
7899 Pope Paschal I
7900 Harry Davis
7901 Saved by the Bell
7902 Schizachyrium condensatum
7903 Swaziland
7904 Lithuania
7905 Belgium
7906 John Bosco
7907 Dances with Wolves
7908 Raiden IV
7909 Super Mario 64
7910 Stir of Echoes
7911 Wind Will Carry Us
7912 Austria
7913 The Girl Who Played with Fire
7914 NFL GameDay 99
7915 Mini Ninjas
7916 Beyond Good & Evil
7917 Jimmie Foxx
7918 Sri Lanka
7919 Calamovilfa longifolia
7920 3rd Rock from the Sun
7921 Christian Ludwig Gerling
7922 Malayala Manorama
7923 FIFA 06
7924 

8200 Green-winged pytilia
8201 white dwarf
8202 French Franc
8203 Sgt. Slaughter
8204 Casuarina cunninghamiana
8205 Clerks
8206 The Untouchables
8207 Toy Story
8208 Finland
8209 Tom Clancy's Ghost Recon
8210 FedEx
8211 A Clockwork Orange
8212 Pope Evaristus
8213 MARBLED SALAMANDER
8214 Museum of New Zealand Te Papa Tongarewa
8215 Poop?-See
8468806_0_4382447409703007384 1 48
Poop?-See
8216 First Templar, TheThe First Templar
22864497_0_8632623712684511496 0 146
First Templar, TheThe First Templar
8217 Dollywaggon Pike
8218 Blue-breasted bee-eater
8219 Southern-Indian-Lake
8220 SEAL SALAMANDER
8221 Return of the Jedi
8222 MALLARD
8223 Thomas Aquinas
8224 COMMON GOLDENEYE
8225 Seatallan
8226 Swaziland
8227 Passiflora tarminiana
8228 Moulin Rouge!
8229 Gem?ldegalerie der Akademie der bildenden K?nste
9834884_0_3871985887467090123 1 278
Gem?ldegalerie der Akademie der bildenden K?nste
8230 Red-cheeked cordonbleu
8231 Yoke
8232 Gull Lake
8233 The Journal of Nutrition
8234 Battlefield: Bad Co

In [31]:
print(result_df.shape)
print(target.shape)
result_df[:20]
# result_df[10:13]

(8415, 4)
(8417, 3)


,TableID,ColumnID,RowID,URI
0,50245608_0_871275842592178099,0,154,http://dbpedia.org/resource/Pearl_Harbor_(film)
1,39107734_2_2329160387535788734,1,32,http://dbpedia.org/resource/Minneapolis-Saint_...
2,22864497_0_8632623712684511496,0,227,http://dbpedia.org/resource/Need_for_Speed:_Ca...
3,66009064_0_9148652238372261251,0,15,http://dbpedia.org/resource/Alex_Rodriguez
4,21362676_0_6854186738074119688,1,75,http://dbpedia.org/resource/King_Kong_(1933_film)
5,50245608_0_871275842592178099,0,186,http://dbpedia.org/resource/School_of_Rock
6,40534006_0_4617468856744635526,1,1,http://dbpedia.org/resource/Lex_Luger
7,36102169_0_7739454799295072814,2,286,http://dbpedia.org/resource/Crash_Bash
8,53822652_0_5767892317858575530,1,141,http://dbpedia.org/resource/The_Hurt_Locker
9,60319454_0_3938426910282115527,0,29,http://dbpedia.org/resource/Lion


In [32]:
# zapisanie submission
# przez evaluator codes można sprawdzić wyniki
result_df.to_csv("D:\Dokumenty\Zastosowanie Technologii Informatycznych\evaluator_codes\evaluator_codes\data\my_submission_films3.csv",
                 index=False,
                 sep=',',
                header=False)

### NIEISTOTNE, TESTY ITP.

In [ ]:
df = pd.read_csv("D:\\Dokumenty\\Zastosowanie Technologii Informatycznych\\data\\CEA_Round1\\13719111_1_5719401842463579519.csv",
                sep=',')
df[:3]

In [ ]:
detect(df.iloc[0:1, 1:2].values[0][0])

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?label
    WHERE { <http://dbpedia.org/resource/Norway> rdfs:label ?label }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)
for result in results["results"]["bindings"]:
    print(result["label"]["value"])

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX ont: <http://dbpedia.org/ontology/>
    SELECT ?a, ?c
    WHERE { 
            ?a ont:location <http://dbpedia.org/resource/Delft> . 
            FILTER regex(?a, "Ikea", "i")
          }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)
# print(result['a'])
# print(result['b'])
# print(result['c'])

# for result in results["results"]["bindings"]:
#     print(type(result))

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sss = "dbr:Game_of_Thrones"
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery('''
ASK {e}
    VALUES (?r) {e} ({source}) {w}
        {e} ?r ?p ?o {w}
        UNION
        {e} ?s ?r ?o {w}
        UNION
        {e} ?s ?p ?r {w}
    {w} 
'''.format(source=sss, e="{", w="}"))
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sss = "dbr:Game_of_Thrones"
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery('''
ASK {
    VALUES (?r) { (dbr:Gra_o_tron) }
        { ?r ?p ?o }
        UNION
        { ?s ?r ?o }
        UNION
        { ?s ?p ?r }
    } 
''')
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print(results)



In [ ]:
SELECT ?r ?p ?o
WHERE {
 VALUES (?r) { (dbr:Cat) }
        { ?r ?p ?o }
        UNION
        { ?s ?r ?o }
        UNION
        { ?s ?p ?r }
}

select ?s
where {
        { ?s rdfs:label "Kot domowy"@pl }
        UNION
        { "Kot domowy"@pl rdfs:label ?s }
} 

SELECT ?r
WHERE {
        { ?r rdfs:label "Kaspisches Meer"@de }

}

In [ ]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    SELECT ?resource
    WHERE { ?resource rdfs:label "Kaspisches Meer"@de }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
# print(results['results']['bindings'][1]['r']['value'])
# results = str(results['results']['bindings'])
print(str(results['results']['bindings']))
if str(results['results']['bindings']).find("http://dbpedia.org/resource/") == -1:
    print("nie ma")
else:
    print('jest')
# for result_dict in results_list:
#     uri = result_dict['resource']['value']
#     match = re.match("http://dbpedia.org/resource/*", uri)
#     if match:
#         print(uri)
#     else:
#         print("NIE")

# print(results['results']['bindings'])

In [ ]:
import re
match = re.match("http://dbpedia.org/resource/*",'http://dbpedia.org/resource/Caspian_Sea', re.M|re.I)
if match:
    print(match.group())